In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/friedman/many"
results_dir_relu = "results/regression/single_layer/relu/friedman/full_regularization"
results_dir_tanh = "results/regression/single_layer/tanh/friedman/full_regularization"
model_names_relu = ["Dirichlet Student T"]
model_names_tanh = ["Dirichlet Student T tanh"]
#model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
#model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh"]


relu_fits = {}
tanh_fits = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    # tanh_fit = get_model_fits(
    #     config=full_config_path,
    #     results_dir=results_dir_tanh,
    #     models=model_names_tanh,
    #     include_prior=False,
    # )
    

    relu_fits[base_config_name] = relu_fit  # use clean key
    #tanh_fits[base_config_name] = tanh_fit  # use clean key
    


In [ ]:
data_dir = f"datasets/friedman/many"
results_dir_relu = "results/regression/single_layer/relu/friedman/full_regularization"
model_names_relu = ["Dirichlet Student T"]



files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    relu_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_relu,
        models=model_names_relu,
        include_prior=False,
    )
    
    relu_fits[base_config_name] = relu_fit  # use clean key
    


In [4]:

from properscoring import crps_ensemble

seed_to_N = {
    1: 100, 3: 100, 4: 100, 5: 100, 6: 100,
    2: 200, 7: 200, 8: 200, 9: 200, 10: 200,
    11: 500, 12: 500, 13: 500, 14: 500, 15: 500
}

def get_N_sigma(seed):
    if seed not in seed_to_N:
        raise ValueError(f"Unsupported seed: {seed}")
    return seed_to_N[seed], 1


def compute_rmse_results(seeds, models, all_fits, get_N_sigma):
    results = []
    posterior_means = []

    for seed in seeds:
        N, sigma = get_N_sigma(seed)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'
        path = f"datasets/friedman/{dataset_key}.npz"

        try:
            data = np.load(path)
            y_test = data["y_test"]
        except FileNotFoundError:
            print(f"[SKIP] File not found: {path}")
            continue

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                output_test = fit.stan_variable("output_test")  # (S, N/5, O)
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = output_test.shape[0]
            rmses = np.zeros(S)

            for i in range(S):
                y_hat = output_test[i]
                rmses[i] = np.sqrt(np.mean((y_hat.squeeze() - y_test.squeeze()) ** 2))
                
            posterior_mean = np.mean(output_test, axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean.squeeze() - y_test.squeeze()) ** 2))

            posterior_means.append({
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'posterior_mean_rmse': posterior_mean_rmse
            })

            for i in range(S):
                results.append({
                    'seed': seed,
                    'N': N,
                    'sigma': sigma,
                    'model': model,
                    'rmse': rmses[i]
                })

    df_rmse = pd.DataFrame(results)
    df_posterior_rmse = pd.DataFrame(posterior_means)

    return df_rmse, df_posterior_rmse



def compute_crps_results(seeds, models, all_fits, get_N_sigma):
    """
    Compute CRPS for each model and dataset using full posterior predictive samples.

    Returns:
        df_crps: DataFrame with model-level CRPS scores.
    """
    model_crps_list = []

    for seed in seeds:
        N, sigma = get_N_sigma(seed)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'
        path = f"datasets/friedman/{dataset_key}.npz"

        try:
            data = np.load(path)
            y_test = data["y_test"].squeeze()  # shape (N_test,)
        except FileNotFoundError:
            print(f"[SKIP] File not found: {path}")
            continue

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                output_test = fit.stan_variable("output_test")  # (S, N_test, 1)
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S, N_test, output_dim = output_test.shape
            assert output_dim == 1, f"Expected output dim = 1, got shape {output_test.shape}"

            preds = output_test.squeeze(-1)  # (S, N_test)

            crps_per_point = crps_ensemble(y_test, preds.T)  # shape (N_test,)
            crps_mean = crps_per_point.mean()

            model_crps_list.append({
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'crps': crps_mean
            })

    df_crps = pd.DataFrame(model_crps_list)

    return df_crps


In [ ]:
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

df_rmse_relu, df_posterior_rmse_relu = compute_rmse_results(
    seeds, model_names_relu, relu_fits, get_N_sigma
)

# df_rmse_tanh, df_posterior_rmse_tanh = compute_rmse_results(
#     seeds, model_names_tanh, tanh_fits, get_N_sigma
# )


In [5]:
seeds = [1, 2]

df_crps_relu = compute_crps_results(
    seeds, model_names_relu, relu_fits, get_N_sigma
)

df_crps_tanh = compute_crps_results(
    seeds, model_names_tanh, tanh_fits, get_N_sigma
)

In [ ]:
df_crps_relu

In [ ]:
df_crps_tanh

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.ticker import FixedLocator

# Combine and tag activation
df_relu = df_rmse_relu.copy()
df_relu["activation"] = "ReLU"

df_tanh = df_rmse_tanh.copy()
df_tanh["activation"] = "tanh"

df_all = pd.concat([df_relu, df_tanh])
df_all["model"] = df_all["model"].str.replace(" tanh", "", regex=False)

# Order of models and activations
model_order = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
activation_order = ["ReLU", "tanh"]

fig, axes = plt.subplots(1, 2, figsize=(18, 7), sharey=True)

for i, N_val in enumerate([100, 200]):
    ax = axes[i]
    df_plot = df_all[(df_all["N"] == N_val)].copy()

    # Use model as x, activation as hue
    sns.boxplot(
        data=df_plot,
        x="model",
        y="rmse",
        hue="activation",
        order=model_order,
        hue_order=activation_order,
        ax=ax
    )

    ax.set_title(f"N = {N_val}")
    ax.set_xlabel("")
    ax.set_ylabel("RMSE")

    ax.grid(True)

    ax.get_legend().remove()

# Add shared legend at top center
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, title="Activation", loc="upper center", ncol=2)

plt.tight_layout(rect=[0, 0, 1, 0.93])
plt.show()


In [6]:
import numpy as np
import pandas as pd
import glob

def compute_true_y_std_from_Xtrain(X_train, sigma, seed):
    x0, x1, x2, x3, x4 = X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3], X_train[:, 4]
    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5 * x4
    )
    rng = np.random.default_rng(seed)
    y = y_clean + rng.normal(0, sigma, size=len(X_train))
    return y.std()

def get_y_std_map_from_X(data_dir="datasets/friedman"):
    y_stds = {}
    for path in glob.glob(f"{data_dir}/*.npz"):
        data = np.load(path)
        X_train = data["X_train"]
        
        # Parse metadata from filename
        fname = path.split("/")[-1].replace(".npz", "")
        parts = fname.split("_")
        N = int(parts[1][1:])
        sigma = float(parts[3][5:])
        seed = int(parts[4][4:])

        y_std = compute_true_y_std_from_Xtrain(X_train, sigma, seed)
        y_stds[fname] = y_std
    return y_stds

# Step 2: Add original-scale RMSE to your DataFrame
def add_original_rmse(df, y_std_map):
    df = df.copy()
    df["dataset_key"] = df.apply(
        lambda row: f"Friedman_N{row.N}_p10_sigma{row.sigma:.2f}_seed{row.seed}", axis=1
    )
    df["rmse_original"] = df.apply(
        lambda row: row.rmse * y_std_map.get(row.dataset_key, np.nan), axis=1
    )
    return df

# Usage
y_std_map = get_y_std_map_from_X("datasets/friedman")
df_rmse_with_orig_relu = add_original_rmse(df_rmse_relu, y_std_map)
df_rmse_with_orig_tanh = add_original_rmse(df_rmse_tanh, y_std_map)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib.ticker import FixedLocator

# Combine and tag activation
df_relu = df_rmse_with_orig_relu.copy()
df_relu["activation"] = "ReLU"

df_tanh = df_rmse_with_orig_tanh.copy()
df_tanh["activation"] = "tanh"

df_all = pd.concat([df_relu, df_tanh])
df_all["model"] = df_all["model"].str.replace(" tanh", "", regex=False)

# Order of models and activations
model_order = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
activation_order = ["ReLU", "tanh"]

fig, axes = plt.subplots(1, 2, figsize=(18, 7), sharey=True)

for i, N_val in enumerate([100, 200]):
    ax = axes[i]
    df_plot = df_all[(df_all["N"] == N_val)].copy()

    # Use model as x, activation as hue
    sns.boxplot(
        data=df_plot,
        x="model",
        y="rmse_original",
        hue="activation",
        order=model_order,
        hue_order=activation_order,
        ax=ax
    )

    ax.set_title(f"N = {N_val}")
    ax.set_xlabel("")
    ax.set_ylabel("RMSE")

    ax.grid(True)

    ax.get_legend().remove()

# Add shared legend at top center
handles, labels = axes[0].get_legend_handles_labels()
fig.legend(handles, labels, title="Activation", loc="upper center", ncol=2)

plt.tight_layout(rect=[0, 0, 1, 0.93])
plt.show()